# Multiclass Prediction and Postprocessing Code for Multiple Images

In [ ]:
# Import necessary libraries
import copy
import glob
import json
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.transforms as transforms
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import shutil
import time
import tifffile as tiff
import cv2
import re


from math import sqrt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.patches import Ellipse
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from pycocotools.coco import COCO
from scipy.optimize import curve_fit
from scipy.spatial import ConvexHull, cKDTree
from scipy.stats import norm
from shapely.geometry import LineString, MultiPoint, Polygon, MultiPolygon
from shapely.ops import unary_union
from skimage import io, measure, morphology
from skimage.draw import polygon, polygon2mask
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from ultralytics import YOLO
import concurrent.futures
import cupy as cp
import torch
import torchvision.transforms.functional as TF
from scipy.spatial.qhull import QhullError

## Model Paths

In [ ]:
# Original models dictionary
models = {
    "FCC_Model": "D:\\Machine_Learning\\CV Script\\Models\\FCC_2024\\weights\\best.pt",
    "BCC_Model": "D:\\Machine_Learning\\CV Script\Models\\BCC_1024\\weights\\best.pt",
    "HCP_Model": "D:\\Machine_Learning\\CV Script\Models\\HCP_1024\\weights\\best.pt",
    "Combined_FCC_BCC_HCP_Model": "D:\\Machine_Learning\\CV Script\\Models\\Combined_FCC_BCC_HCP_1024\\weights\\best.pt",
    "Combined_FCC_BCC_Model": "D:\\Machine_Learning\\CV Script\\Models\\Combined_FCC_BCC_1024\\weights\\best.pt",
    "Combined_FCC_HCP_Model": "D:\\Machine_Learning\\CV Script\\Models\\Combined_FCC_HCP_1024\\weights\\best.pt",
    "Combined_HCP_BCC_Model": "D:\\Machine_Learning\\CV Script\\Models\\Combined_BCC_HCP_1024\\weights\\best.pt",
    "SwiggleSlip_Model": "D:\\Machine_Learning\\CV Script\\Models\\SwiggleSlip_1024\\weights\\best.pt"
}

# Shortcut mapping
shortcut_mapping = {
    "FCC": "FCC_Model",
    "BCC": "BCC_Model",
    "HCP": "HCP_Model",
    "FBH": "Combined_FCC_BCC_HCP_Model",
    "FB": "Combined_FCC_BCC_Model",
    "FH": "Combined_FCC_HCP_Model",
    "BH": "Combined_HCP_BCC_Model",
    "Swiggle": "SwiggleSlip_Model"
}

# Function to retrieve model path by shortcut
def get_model_path(shortcut):
    model_key = shortcut_mapping.get(shortcut)
    if model_key:
        return models[model_key]
    else:
        raise ValueError(f"Shortcut '{shortcut}' not recognized.")


# Set up Image Paths

In [ ]:
# Define initial paths and variables
image_paths =  [
                " "
                ]

base_output_folder = ' '  # Base output folder for all results

model_path = get_model_path(" ") # FCC, BCC, HCP or Combinations abbreviated as FBH, FB, FH, or BH.
conversion_factor = 22.46 # To convert intensity to nanometers


# Functions

In [ ]:
#Functions to do Initial Predictions
def apply_previous_predictions(image_path, previous_json_path, output_folder, dilation_distance=5):

    # Check if the previous predictions JSON file exists
    if os.path.exists(previous_json_path):
        print(f"Loading previous predictions from {previous_json_path}")

        # Load the COCO JSON file
        with open(previous_json_path, 'r') as f:
            coco_data = json.load(f)

        # Load the image
        image = tiff.imread(image_path)
        image_shape = image.shape

        # Create a blank mask with the same shape as the image
        if image.ndim == 2:  # Grayscale
            mask = np.zeros_like(image, dtype=np.uint8)
            fill_value = 0  # Zero value for masking
        else:  # RGB
            mask = np.zeros((image_shape[0], image_shape[1], 3), dtype=np.uint8)
            fill_value = (0, 0, 0)  # Zero value for masking

        print(f"Image shape: {image_shape}")
        print(f"Number of annotations: {len(coco_data['annotations'])}")

        # Draw the masks on the blank image
        for annotation in coco_data["annotations"]:
            original_segmentation = annotation["segmentation"][0]  # Assuming one polygon per annotation
            dilated_segmentation = dilate_polygon(original_segmentation, dilation_distance, image_shape)
            if dilated_segmentation:
                rr, cc = polygon(np.array(dilated_segmentation)[:, 1], np.array(dilated_segmentation)[:, 0], image_shape)
                mask[rr, cc] = 1  # Mark the mask region

        print("Mask created. Applying mask to the original image...")

        # Apply the mask to the original image
        if image.ndim == 2:
            masked_image = np.where(mask == 1, fill_value, image)
        else:
            masked_image = np.where(mask == 1, fill_value, image)

        print("Mask applied. Saving the masked image...")

        # Construct output file name
        base_name = os.path.basename(image_path)
        new_file_name = f"masked_{base_name}"
        output_path = os.path.join(output_folder, new_file_name)

        # Save the masked image in the same format
        tiff.imwrite(output_path, masked_image)

        print(f"Masked image saved to {output_path}")

        return output_path
    print(f"No previous predictions found for {image_path}")
    return image_path

def preprocess(image_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Extract filename from the image path
    filename = os.path.basename(image_path)
    
    # Add '_prepro' before the file extension in the filename
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_prepro{ext}"

    # Construct the save path
    save_path = os.path.join(output_folder, new_filename)

    # Read the image
    img = io.imread(image_path)
    img[np.isnan(img)] = 0
    img = (img * 255).clip(0, 255)
    io.imsave(save_path, img)

    return save_path

def pad_image(image_path, output_folder, min_padding=256):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    base_name = os.path.basename(image_path)
    new_file_name = os.path.splitext(base_name)[0] + "_padded" + os.path.splitext(base_name)[1]
    output_path = os.path.join(output_folder, new_file_name)

    with Image.open(image_path) as img:
        width, height = img.size
        padded_width = width + 2 * min_padding
        padded_height = height + 2 * min_padding

        new_width = math.ceil(padded_width / 1024) * 1024
        new_height = math.ceil(padded_height / 1024) * 1024

        new_img = Image.new("RGB", (new_width, new_height), (0, 0, 0))
        new_img.paste(img, ((new_width - width) // 2, (new_height - height) // 2))

        new_img.save(output_path)
    
    top_padding = (new_height - height) // 2
    left_padding = (new_width - width) // 2
    return output_path, top_padding, top_padding, left_padding, left_padding

def slice_image_to_1024(image_path, save_path, counter, shift_distance):
    with Image.open(image_path) as img:
        width, height = img.size

        # Calculate the number of shifts horizontally and vertically
        num_shifts_x = width // 1024
        num_shifts_y = height // 1024

        # Calculate shift based on counter, wrapping around using modulo arithmetic
        shift_x = (counter % num_shifts_x) * shift_distance
        shift_y = (counter // num_shifts_x) * shift_distance

        # Ensure the shift does not exceed the image boundaries
        shift_x = min(shift_x, width - 1024)
        shift_y = min(shift_y, height - 1024)

        # Update save path to include counter
        save_path_with_counter = os.path.join(save_path, f"slices_{counter}")
        if not os.path.exists(save_path_with_counter):
            os.makedirs(save_path_with_counter)

        # Calculate the number of slices in both dimensions
        cols = (width - shift_x) // 1024
        rows = (height - shift_y) // 1024

        # Slice the image and save each piece
        for row in range(rows):
            for col in range(cols):
                left = col * 1024 + shift_x
                upper = row * 1024 + shift_y
                right = left + 1024
                lower = upper + 1024

                slice_img = img.crop((left, upper, right, lower))

                if slice_img.size == (1024, 1024):
                    slice_img.save(os.path.join(save_path_with_counter, f"{left}_{upper}.jpg"))
    
    return save_path_with_counter

def mask_to_polygon(mask, image_size, border_threshold=128, min_size=100):
    
    # Label the mask to identify distinct regions
    labeled_mask, num_labels = measure.label(mask, background=0, return_num=True)

    # If no regions are found, return None
    if num_labels == 0:
        return None

    # Keep only the largest continuous region
    largest_region = morphology.remove_small_objects(labeled_mask, min_size=min_size)
    largest_region_mask = largest_region > 0

    # Check the size of the largest region
    if np.sum(largest_region_mask) < min_size:
        return None

    # Find contours from the binary mask of the largest region
    contours = measure.find_contours(largest_region_mask, 0.5)

    # Convert contours to polygon format and compute convex hull if needed
    polygons = []
    for contour in contours:
        contour = np.flip(contour, axis=1)  # Flip from (row, col) to (x, y) format
        contour = contour.round().astype(int)

        # Check if contour is valid for convex hull computation
        if len(np.unique(contour, axis=0)) < 3:
            # Contour is invalid for convex hull, skip to next contour
            continue

        # Calculate the area of the original polygon
        try:
            original_area = ConvexHull(contour).volume
        except QhullError:
            # If ConvexHull computation fails, skip to next contour
            continue

        # Calculate the convex hull of the polygon
        hull = ConvexHull(contour).vertices
        convex_hull_polygon = contour[hull]

        # Calculate the area of the convex hull
        try:
            convex_hull_area = ConvexHull(convex_hull_polygon).volume
        except QhullError:
            # If ConvexHull computation fails, use the original contour
            polygons.append(contour.flatten().tolist())
            continue

        # Compare areas and decide whether to keep the convex hull
        if (convex_hull_area - original_area) / original_area < 0.1:
            polygons.append(convex_hull_polygon.flatten().tolist())
        else:
            polygons.append(contour.flatten().tolist())

    return polygons

def calculate_area_and_bbox(polygon):
    
    # Convert the flattened list to a 2D array of points
    poly_array = np.array(polygon).reshape(-1, 2)

    # Calculate bounding box
    x_min = np.min(poly_array[:, 0])
    x_max = np.max(poly_array[:, 0])
    y_min = np.min(poly_array[:, 1])
    y_max = np.max(poly_array[:, 1])
    bbox = [int(x_min), int(y_min), int(x_max - x_min), int(y_max - y_min)]

    # Calculate area using Shoelace formula
    x = poly_array[:, 0]
    y = poly_array[:, 1]
    area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    area = int(area)

    return area, bbox

def process_images(input_folder, output_folder, model, counter, min_size, area_threshold):
    coco_output = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 1, "name": "Class_0"},
            {"id": 2, "name": "Class_1"}
        ]
    }

    annotation_id = 1

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.png', '.tif', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            with Image.open(image_path) as img:
                width, height = img.size

                image_id = len(coco_output["images"]) + 1
                coco_output["images"].append({
                    "id": int(image_id),
                    "width": int(width),
                    "height": int(height),
                    "file_name": filename
                })

                results = model(img, imgsz=1024, conf=0.7, device=[0])
                
                for r in results:
                    if r.masks is not None and hasattr(r.masks, 'data'):
                        masks_numpy = [mask.cpu().numpy() for mask in r.masks.data]

                        cls_indices = r.boxes.cls.cpu().numpy().astype(int) if hasattr(r.boxes, 'cls') else [0]*len(masks_numpy)

                        for i, mask in enumerate(masks_numpy):
                            class_idx = cls_indices[i]
                            category_id = int(class_idx + 1)
                            polygon = mask_to_polygon(
                                mask, 
                                image_size=(width, height), 
                                border_threshold=0, 
                                min_size=min_size
                            )

                            if polygon is not None:
                                for poly in polygon:
                                    area, bbox = calculate_area_and_bbox(poly)
                                    # Cast area and bbox values to Python int
                                    area = int(area)
                                    bbox = [int(val) for val in bbox]

                                    # Also ensure polygon coordinates are all ints
                                    poly = [int(coord) for coord in poly]

                                    if area >= area_threshold:
                                        coco_output["annotations"].append({
                                            "id": int(annotation_id),
                                            "image_id": int(image_id),
                                            "category_id": int(category_id),
                                            "segmentation": [poly],
                                            "area": int(area),
                                            "bbox": bbox,
                                            "iscrowd": 0
                                        })
                                        annotation_id += 1

    output_json = os.path.join(output_folder, f"slice_predictions_{counter}.json")
    with open(output_json, 'w') as f:
        json.dump(coco_output, f)  # Should now serialize without error

    return output_json

def convert_coco_to_large_image(coco_file, output_folder, padded_image_path, counter=0):
    # Load the large image to get its size
    with Image.open(padded_image_path) as large_img:
        large_image_size = large_img.size

    # Extract the file name of the large image
    large_image_file_name = os.path.basename(padded_image_path)

    # Load the COCO data
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)

    # Prepare the large image annotations structure
    large_image_annotations = {
        "images": [{"id": 1, "width": large_image_size[0], "height": large_image_size[1], "file_name": large_image_file_name}],
        "annotations": [],
        "categories": coco_data["categories"]
    }

    annotation_id = 1
    for annotation in coco_data["annotations"]:
        image_id = annotation["image_id"]
        image_info = next((img for img in coco_data["images"] if img["id"] == image_id), None)
        if image_info:
            # Extract x, y offsets from the image name
            # The slicing process produces filenames like "left_top.jpg"
            # Ensure the file name contains the offsets as intended.
            # Typically, the original code expected filenames like "X_Y.jpg" where X and Y are integers.
            # If so:
            x_offset, y_offset = map(int, image_info["file_name"].split('.')[0].split('_'))

            seg = annotation["segmentation"]

            # Ensure segmentation is flat: some COCO formats store polygons as [[x0,y0,x1,y1,...]]
            # If there's only one polygon and it's nested, flatten it:
            if len(seg) == 1 and isinstance(seg[0], list):
                seg = seg[0]

            # Now seg should be a flat list of coordinates: [x0, y0, x1, y1, ...]
            # Add offsets to each coordinate
            # Convert coord to int to avoid JSON serialization issues later
            adjusted_seg = [int(coord + (x_offset if i % 2 == 0 else y_offset)) for i, coord in enumerate(seg)]

            # Calculate area and bbox (make sure these are ints)
            area, bbox = calculate_area_and_bbox(adjusted_seg)
            area = int(area)
            bbox = [int(val) for val in bbox]

            # Create new annotation for the large image
            large_image_annotations["annotations"].append({
                "id": annotation_id,
                "image_id": 1,
                "category_id": int(annotation["category_id"]),
                "segmentation": [adjusted_seg],
                "area": area,
                "bbox": bbox,
                "iscrowd": int(annotation["iscrowd"])
            })
            annotation_id += 1

    # Construct output file name with counter
    output_file_path = os.path.join(output_folder, f"full_predictions_{counter}.json")

    # Save the adjusted annotations as a COCO JSON file
    with open(output_file_path, 'w') as f:
        json.dump(large_image_annotations, f)

    return output_file_path

def dilate_polygon(polygon, dilation_distance, image_shape):
    # Ensure the polygon is in the correct format: list of points where each point is a tuple (x, y)
    polygon = [(polygon[i], polygon[i + 1]) for i in range(0, len(polygon), 2)]

    # Create an empty mask
    mask = np.zeros(image_shape, dtype=np.uint8)

    # Draw the polygon on the mask
    cv2.fillPoly(mask, [np.array(polygon, dtype=np.int32)], 1)

    # Create a kernel for dilation
    kernel = np.ones((dilation_distance*2+1, dilation_distance*2+1), np.uint8)

    # Dilate the mask
    dilated_mask = cv2.dilate(mask, kernel)

    # Find contours in the dilated mask
    contours, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assuming the largest contour is the dilated polygon
    if contours:
        dilated_contour = contours[0]
        # Flatten the dilated contour to a list of points
        dilated_polygon = dilated_contour.reshape((-1, 2)).tolist()
    else:
        dilated_polygon = []

    return dilated_polygon

def mask_image_with_coco_masks(image_path, coco_json_path, output_folder, counter, dilation_distance=5):
    # Load the COCO JSON file
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Load the image
    image = Image.open(image_path)
    image_shape = (image.height, image.width)
    draw = ImageDraw.Draw(image)

    # Iterate over annotations and apply each mask
    for annotation in coco_data["annotations"]:
        original_segmentation = annotation["segmentation"][0]  # Assuming one polygon per annotation

        # Dilate the polygon
        dilated_segmentation = dilate_polygon(original_segmentation, dilation_distance, image_shape)

        # Flatten the list of tuples into a flat list of coordinates
        flat_dilated_segmentation = [coord for point in dilated_segmentation for coord in point]

        # Draw the dilated mask with black color
        draw.polygon(flat_dilated_segmentation, fill=(0, 0, 0))

    # Construct output file name with counter
    base_name = os.path.basename(image_path)
    new_file_name = f"masked_{counter}" + os.path.splitext(base_name)[1]
    output_path = os.path.join(output_folder, new_file_name)

    # Save the masked image
    image.save(output_path)

    return output_path

def compile_coco_json(output_folder, output_json, image_name):
    compiled_annotations = {
        "images": [],
        "annotations": [],
        "categories": []  # This should be populated based on your categories
    }

    # Get image dimensions
    image_path = os.path.join(output_folder, image_name)
    with Image.open(image_path) as img:
        width, height = img.size

    compiled_annotations["images"].append({"id": 1, "file_name": image_name, "width": width, "height": height})

    annotation_id = 1
    for json_file in glob.glob(os.path.join(output_folder, "full_predictions_*.json")):
        with open(json_file, 'r') as f:
            data = json.load(f)
            # Assuming categories are the same across all files and already populated
            for annotation in data["annotations"]:
                # Update annotation ID and set image_id to 1
                annotation["id"] = annotation_id
                annotation["image_id"] = 1
                compiled_annotations["annotations"].append(annotation)
                annotation_id += 1

            # If categories are not populated, update them from the first file
            if not compiled_annotations["categories"]:
                compiled_annotations["categories"] = data["categories"]

    output_path = os.path.join(output_folder, output_json)

    # Save the compiled annotations to a single JSON file
    with open(output_path, 'w') as f:
        json.dump(compiled_annotations, f)
    
    return output_path

def visualize_coco_masks(image_path, coco_json_path, output_path, figure_size=(50, 50), font_size=20):
    # Load the COCO JSON file
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Define colors per category_id
    # Adjust these RGB tuples for different classes
    category_colors = {
        1: (255, 0, 0),   # Red for Class_0
        2: (0, 255, 0)    # Green for Class_1
    }

    # Load the image
    image = Image.open(image_path)
    image = image.convert('RGB')
    draw = ImageDraw.Draw(image)

    # Load a font for drawing text
    try:
        font = ImageFont.truetype("arial.ttf", size=font_size)
    except IOError:
        font = ImageFont.load_default()

    # Iterate over annotations and draw each segmentation
    for annotation in coco_data["annotations"]:
        segmentation = annotation["segmentation"][0]  # Assuming one polygon per annotation
        cat_id = annotation["category_id"]
        color = category_colors.get(cat_id, (255, 255, 255))  # Default to white if not found

        # Draw the segmentation polygon
        draw.polygon(segmentation, outline=color, fill=None)

        # Calculate the centroid of the mask
        centroid_x = sum(segmentation[::2]) / (len(segmentation) / 2)
        centroid_y = sum(segmentation[1::2]) / (len(segmentation) / 2)

        # Draw the index of the mask and category ID near the centroid
        label_text = f"ID:{annotation['id']} C:{cat_id}"
        draw.text((centroid_x + 5, centroid_y), label_text, fill=color, font=font)

    # Display the image in a larger figure
    plt.figure(figsize=figure_size)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # Save the image with masks
    image.save(output_path)

def visualize_coco_masks_bbox(image_path, coco_json_path, output_path, figure_size=(50, 50), font_size=20):
    # Load the COCO JSON file
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Define colors per category_id
    category_colors = {
        1: (255, 0, 0),   # Red for Class_0
        2: (0, 255, 0)    # Green for Class_1
    }

    # Load the image
    image = Image.open(image_path)
    image = image.convert('RGB')
    draw = ImageDraw.Draw(image)

    # Load a font for drawing text
    try:
        font = ImageFont.truetype("arial.ttf", size=font_size)
    except IOError:
        font = ImageFont.load_default()

    # Iterate over annotations and draw each segmentation and bounding box
    for annotation in coco_data["annotations"]:
        segmentation = annotation["segmentation"][0]  # Assuming one polygon per annotation
        bbox = annotation["bbox"]
        cat_id = annotation["category_id"]
        color = category_colors.get(cat_id, (255, 255, 255))

        # Draw the segmentation polygon
        draw.polygon(segmentation, outline=color, fill=None)

        # Draw the bounding box
        bbox_coords = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
        draw.rectangle(bbox_coords, outline=color, width=2)

        # Calculate the centroid of the mask for placing the index text
        centroid_x = sum(segmentation[::2]) / (len(segmentation) / 2)
        centroid_y = sum(segmentation[1::2]) / (len(segmentation) / 2)

        # Draw the annotation ID and category near the centroid
        label_text = f"ID:{annotation['id']} C:{cat_id}"
        draw.text((centroid_x + 5, centroid_y), label_text, fill=color, font=font)

    # Display the image in a larger figure
    plt.figure(figsize=figure_size)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # Save the image with masks and bounding boxes
    image.save(output_path)

def visualize_coco_masks_bbox_black(image_path, coco_json_path, output_path, figure_size=(50, 50), font_size=20, black=False, bbox=False):
    # Load the COCO JSON file
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Define colors per category_id
    category_colors = {
        1: (255, 0, 0),   # Red for Class_0
        2: (0, 255, 0)    # Green for Class_1
    }

    # Load the image
    image = Image.open(image_path)
    image = image.convert('RGB')

    # If black background requested
    if black:
        # Create a black canvas of the same size
        black_img = Image.new('RGB', image.size, (0, 0, 0))
        image = black_img

    draw = ImageDraw.Draw(image)

    # Load a font for drawing text
    try:
        font = ImageFont.truetype("arial.ttf", size=font_size)
    except IOError:
        font = ImageFont.load_default()

    # Iterate over annotations and draw each segmentation
    for annotation in coco_data["annotations"]:
        segmentation = annotation["segmentation"][0]  # Assuming one polygon per annotation
        cat_id = annotation["category_id"]
        color = category_colors.get(cat_id, (255, 255, 255))

        # Draw the segmentation polygon
        draw.polygon(segmentation, outline=color, fill=None)

        if bbox:
            # Draw the bounding box if requested
            bbox_coords = annotation["bbox"]
            bbox_coords = [bbox_coords[0], bbox_coords[1], bbox_coords[0] + bbox_coords[2], bbox_coords[1] + bbox_coords[3]]
            draw.rectangle(bbox_coords, outline=color, width=2)

        # Calculate centroid for placing ID and category info
        centroid_x = sum(segmentation[::2]) / (len(segmentation) / 2)
        centroid_y = sum(segmentation[1::2]) / (len(segmentation) / 2)
        label_text = f"ID:{annotation['id']} C:{cat_id}"
        draw.text((centroid_x + 5, centroid_y), label_text, fill=color, font=font)

    # Display the image in a larger figure
    plt.figure(figsize=figure_size)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # Save the image
    image.save(output_path)


In [ ]:
#Function to Clean Annotations
def clean_annotations(image_path, json_path, output_path, intensity_threshold=1):
    #Load the image using PIL and convert to numpy array for processing
    try:
        pil_image = Image.open(image_path).convert('L')
        image = np.array(pil_image)
    except Exception as e:
        raise FileNotFoundError(f"Failed to load image at path {image_path}: {str(e)}")

    # Load the JSON data
    with open(json_path, 'r') as file:
        data = json.load(file)
    annotations = data.get('annotations', [])

    # Calculate the average intensity of the entire image
    average_intensity_image = np.mean(image)
    print("Average intensity of the entire image:", average_intensity_image)

    # Dictionary for class-specific intensity thresholds, if desired.
    # Adjust values if some classes require different minimum intensity thresholds.
    class_thresholds = {
        1: intensity_threshold,  # Class_0
        2: intensity_threshold   # Class_1
    }

    filtered_annotations = []
    for annotation in annotations:
        if 'segmentation' not in annotation or not isinstance(annotation['segmentation'][0], list):
            continue

        category_id = annotation.get('category_id', 1)  # Default to 1 if not present
        # Print category_id for debugging/confirmation
        print(f"Processing Annotation ID {annotation['id']} with Category ID {category_id}")

        # Create a mask for the current annotation
        mask = np.zeros(image.shape, dtype=np.uint8)
        for polygon in annotation['segmentation']:
            points = np.array(polygon, dtype=np.int32).reshape((-1, 1, 2))
            cv2.fillPoly(mask, [points], 255)

        if np.count_nonzero(mask) == 0:
            print(f"Annotation ID {annotation['id']} has an empty mask. Check polygon coordinates.")
            continue

        # Extract the pixel values where the mask is applied
        masked_pixels = image[mask > 0]
        if masked_pixels.size == 0:
            print(f"Annotation ID {annotation['id']} has a zero-area mask after application.")
            continue

        # Calculate the average intensity of the pixels within the mask
        average_intensity_mask = np.mean(masked_pixels)

        # Use class-specific intensity threshold if available
        category_threshold = class_thresholds.get(category_id, intensity_threshold)

        # Compare the mask's average intensity to the image's average intensity scaled by threshold
        if average_intensity_mask >= average_intensity_image * category_threshold:
            filtered_annotations.append(annotation)
        else:
            print(f"Annotation ID {annotation['id']} removed due to low intensity for Category {category_id}.")

    # Update the JSON data with filtered annotations
    data['annotations'] = filtered_annotations

    # Save the cleaned data to a JSON file
    with open(output_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"Cleaned annotations saved to {output_path}")

def convert_seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return int(hours), int(minutes), seconds


In [ ]:
#Functions for Merging
def calculate_polygon_area(polygon):
    x = [point[0] for point in polygon]
    y = [point[1] for point in polygon]
    return 0.5 * abs(sum(x[i] * y[(i + 1) % len(polygon)] for i in range(len(polygon))) -
                     sum(y[i] * x[(i + 1) % len(polygon)] for i in range(len(polygon))))

def calculate_centroid(points):
    area = C_x = C_y = 0.0
    for i in range(len(points)):
        j = (i + 1) % len(points)
        cross = points[i][0] * points[j][1] - points[j][0] * points[i][1]
        area += cross
        C_x += (points[i][0] + points[j][0]) * cross
        C_y += (points[i][1] + points[j][1]) * cross
    area *= 0.5
    if area != 0:
        C_x /= (6 * area)
        C_y /= (6 * area)
    return (C_x, C_y)

def calculate_principal_axes(points):
    points_array = np.array(points)
    centroid = points_array.mean(axis=0)
    centered_points = points_array - centroid
    covariance_matrix = np.cov(centered_points.T)  # Note the transpose for correct dimensionality
    eigenvalues, _ = np.linalg.eig(covariance_matrix)
    axes_lengths = np.sqrt(eigenvalues)
    major_axis_length, minor_axis_length = np.sort(axes_lengths)[::-1]
    return major_axis_length, minor_axis_length

def calculate_orientation(points):
    points_array = np.array(points)
    centroid = points_array.mean(axis=0)
    centered_points = points_array - centroid
    pca = PCA(n_components=2)
    pca.fit(centered_points)
    major_axis_vector = pca.components_[0]  # First principal component
    angle = np.arctan2(major_axis_vector[1], major_axis_vector[0])
    return np.degrees(angle) % 360

def create_polygon_from_points(points):
    return Polygon(points)

def calculate_intersection_area(poly1, poly2):
    intersection = poly1.intersection(poly2)
    return intersection.area

def calculate_line_overlap_length(poly1, poly2):
    # Ensure polygons are valid and simplifying them might resolve some edge cases
    poly1 = poly1.simplify(0.01, preserve_topology=True)
    poly2 = poly2.simplify(0.01, preserve_topology=True)

    centroid1 = poly1.centroid.coords[0]
    centroid2 = poly2.centroid.coords[0]
    line = LineString([centroid1, centroid2])

    intersection_line = line.intersection(poly1.intersection(poly2))
    
    # If there's no intersection or if it's a Point, there's no overlap length
    if intersection_line.is_empty or intersection_line.geom_type == 'Point':
        return 0

    # Make sure we're calculating the length of a LineString
    if isinstance(intersection_line, LineString):
        return intersection_line.length
    elif hasattr(intersection_line, 'geoms'):  # It's a MultiLineString or similar
        return sum(geom.length for geom in intersection_line.geoms)
    else:
        # Log unexpected geometry types
        print(f"Unexpected geometry type: {intersection_line.geom_type}")
        return 0

def euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def angle_difference(angle1, angle2):
    direct_diff = abs(angle1 - angle2) % 360
    reverse_diff = abs(180 - direct_diff)
    return direct_diff, reverse_diff

def convex_hull_area(points1, points2):
    points = np.vstack((points1, points2))
    hull = ConvexHull(points)
    return hull.volume

def combine_areas(area1, area2):
    return area1 + area2

def plot_orientation(ax, centroid, length, angle, color):
    # This function adds an orientation line to the plot based on centroid, length, and angle
    rad = np.deg2rad(angle)
    x = [centroid[0], centroid[0] + length * np.cos(rad)]
    y = [centroid[1], centroid[1] + length * np.sin(rad)]
    ax.plot(x, y, color=color, linestyle='dashed')

def visualize_annotations(ann1, ann2):
    fig, ax = plt.subplots()
    
    # Plot the polygons
    poly1 = create_polygon_from_points(ann1['polygon_points'])
    poly2 = create_polygon_from_points(ann2['polygon_points'])
    
    x1, y1 = poly1.exterior.xy
    x2, y2 = poly2.exterior.xy
    
    ax.fill(x1, y1, alpha=0.5, fc='red', label=f'Annotation {ann1["annotation_id"]}')
    ax.fill(x2, y2, alpha=0.5, fc='blue', label=f'Annotation {ann2["annotation_id"]}')
    
    # Plot orientation vectors
    plot_orientation(ax, ann1['centroid'], ann1['principal_axes_lengths']['major_axis_length'], ann1['orientation'], 'red')
    plot_orientation(ax, ann2['centroid'], ann2['principal_axes_lengths']['major_axis_length'], ann2['orientation'], 'blue')
    
    # Settings and labels
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    ax.set_title('Annotation Comparison')
    ax.legend()

    plt.axis('equal')
    plt.show()

def process_coco_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    annotations = data.get('annotations', [])
    results = []

    # First, parse all annotations and calculate necessary geometric properties
    for annotation in annotations:
        if 'segmentation' in annotation:
            # Store category_id
            category_id = annotation.get('category_id', 1)
            for polygon_data in annotation['segmentation']:
                if isinstance(polygon_data, list):
                    # Flatten polygon data if nested
                    if all(isinstance(elem, list) for elem in polygon_data):
                        polygon_data = [coord for sublist in polygon_data for coord in sublist]
                    polygon_points = [(polygon_data[i], polygon_data[i + 1]) for i in range(0, len(polygon_data), 2)]
                    area = calculate_polygon_area(polygon_points)
                    centroid = calculate_centroid(polygon_points)
                    orientation = calculate_orientation(polygon_points)
                    major_axis_length, minor_axis_length = calculate_principal_axes(polygon_points)
                    result = {
                        'annotation_id': annotation['id'],
                        'category_id': category_id,  # Store category_id
                        'area': area,
                        'bbox': annotation['bbox'],
                        'centroid': centroid,
                        'orientation': orientation,
                        'principal_axes_lengths': {
                            'major_axis_length': major_axis_length,
                            'minor_axis_length': minor_axis_length
                        },
                        'polygon_points': polygon_points,
                        'nearby_ids': [],
                        'segmentation': annotation['segmentation']
                    }
                    results.append(result)

    # Calculate proximity of annotations
    for i, ann1 in enumerate(results):
        for j, ann2 in enumerate(results):
            if i != j:
                distance = euclidean_distance(ann1['centroid'], ann2['centroid'])
                if distance < 3 * ann1['principal_axes_lengths']['major_axis_length']:
                    ann1['nearby_ids'].append(ann2['annotation_id'])

    return results

def merge_annotations(ann1, ann2):
    # Combine polygon points
    all_points = ann1['polygon_points'] + ann2['polygon_points']

    multipoint = MultiPoint(all_points)
    convex_hull = multipoint.convex_hull

    if convex_hull.geom_type == 'Polygon':
        hull_points_list = list(convex_hull.exterior.coords)
    else:
        # If not a polygon, fallback to all_points
        hull_points_list = all_points

    new_area = calculate_polygon_area(hull_points_list)
    new_centroid = calculate_centroid(hull_points_list)
    new_orientation = calculate_orientation(hull_points_list)
    new_major_axis_length, new_minor_axis_length = calculate_principal_axes(hull_points_list)
    bbox = [int(coord) for coord in convex_hull.bounds]

    # Determine category_id:
    # Assuming we only merge annotations of the same category.
    # If not, add logic to determine the resulting category.
    category_id = ann1['category_id']

    ancestors = set(ann1.get('ancestors', [ann1['annotation_id']]) + ann2.get('ancestors', [ann2['annotation_id']]))

    return {
        'annotation_id': f"merged-{ann1['annotation_id']}-{ann2['annotation_id']}",
        'category_id': category_id,
        'area': new_area,
        'centroid': new_centroid,
        'orientation': new_orientation,
        'principal_axes_lengths': {
            'major_axis_length': new_major_axis_length,
            'minor_axis_length': new_minor_axis_length
        },
        'polygon_points': hull_points_list,
        'segmentation': [coord for point in hull_points_list for coord in point],
        'bbox': bbox,
        'ancestors': list(ancestors)
    }

def check_compatibility_single(ann1, ann2, categories_to_merge=None):
    # If categories_to_merge is provided, ensure both annotations are in these categories
    if categories_to_merge is not None:
        if ann1['category_id'] not in categories_to_merge or ann2['category_id'] not in categories_to_merge:
            return False  # Don't merge if either annotation's category is not in allowed list

    # Prevent merging if any ancestors are the same
    if set(ann1.get('ancestors', [ann1['annotation_id']])) & set(ann2.get('ancestors', [ann2['annotation_id']])):
        return False
    
    poly1 = Polygon(ann1['polygon_points'])
    poly2 = Polygon(ann2['polygon_points'])

    intersection = poly1.intersection(poly2)
    intersection_area = intersection.area

    overlap_percent_ann1 = intersection_area / ann1['area'] if ann1['area'] > 0 else 0
    overlap_percent_ann2 = intersection_area / ann2['area'] if ann2['area'] > 0 else 0

    if overlap_percent_ann1 >= 0.5 or overlap_percent_ann2 >= 0.5:
        return True

    major_axis_length_ann1 = ann1['principal_axes_lengths']['major_axis_length']
    minor_axis_length_ann1 = ann1['principal_axes_lengths']['minor_axis_length']
    diagonal_ratio_ann1 = minor_axis_length_ann1 / major_axis_length_ann1 if major_axis_length_ann1 > 0 else 0

    major_axis_length_ann2 = ann2['principal_axes_lengths']['major_axis_length']
    minor_axis_length_ann2 = ann2['principal_axes_lengths']['minor_axis_length']
    diagonal_ratio_ann2 = minor_axis_length_ann2 / major_axis_length_ann2 if major_axis_length_ann2 > 0 else 0

    ignore_orientation = diagonal_ratio_ann1 > 0.33 or diagonal_ratio_ann2 > 0.33

    overlap_length = calculate_line_overlap_length(poly1, poly2)
    dist = euclidean_distance(ann1['centroid'], ann2['centroid'])
    direct_diff, reverse_diff = angle_difference(ann1['orientation'], ann2['orientation'])
    orientation_diff = min(direct_diff, reverse_diff)
    convex_hull_ratio = convex_hull_area(np.array(ann1['polygon_points']), np.array(ann2['polygon_points'])) / combine_areas(ann1['area'], ann2['area'])

    return (dist < 3 * max(major_axis_length_ann1, major_axis_length_ann2) and
            (ignore_orientation or orientation_diff <= 5) and
            convex_hull_ratio < 1.3 and
            dist > ((major_axis_length_ann1 + major_axis_length_ann2 - overlap_length)))

def progressive_merge_annotations(annotations, categories_to_merge=None):
    # If categories_to_merge is not specified, assume all categories are mergable
    # Otherwise, merging will only happen among these categories.
    if categories_to_merge is None:
        categories_to_merge = []

    points = np.array([ann['centroid'] for ann in annotations])
    kd_tree = cKDTree(points)

    remaining_annotations = copy.deepcopy(annotations)
    merged = True

    while merged:
        merged = False
        new_annotations = []
        new_points = []
        skip_indices = set()

        for i, ann1 in enumerate(remaining_annotations):
            if i in skip_indices:
                continue

            # Only attempt merges if ann1 is in the allowed categories
            if ann1['category_id'] in categories_to_merge:
                # Query close neighbors
                indices = kd_tree.query(points[i], k=50)[1]
                indices = [index for index in indices if index < len(remaining_annotations) and index not in skip_indices and index != i]

                for j in indices:
                    ann2 = remaining_annotations[j]

                    if check_compatibility_single(ann1, ann2, categories_to_merge=categories_to_merge):
                        merged_annotation = merge_annotations(ann1, ann2)
                        # Update properties
                        merged_annotation['segmentation'] = [coord for point in merged_annotation['polygon_points'] for coord in point]
                        merged_annotation['centroid'] = calculate_centroid(merged_annotation['polygon_points'])
                        x_coords, y_coords = zip(*merged_annotation['polygon_points'])
                        merged_annotation['bbox'] = [
                            int(min(x_coords)),
                            int(min(y_coords)),
                            int(max(x_coords) - min(x_coords)),
                            int(max(y_coords) - min(y_coords))
                        ]
                        merged_annotation['nearby_ids'] = list(set(ann1.get('nearby_ids', []) + ann2.get('nearby_ids', [])) - {ann1['annotation_id'], ann2['annotation_id']})
                        
                        new_annotations.append(merged_annotation)
                        new_points.append([
                            merged_annotation['bbox'][0] + merged_annotation['bbox'][2] / 2, 
                            merged_annotation['bbox'][1] + merged_annotation['bbox'][3] / 2
                        ])

                        skip_indices.update({i, j})
                        merged = True
                        break

                if i not in skip_indices:
                    # ann1 not merged
                    new_annotations.append(ann1)
                    new_points.append(points[i])
            else:
                # ann1 category not in merge list, skip merging and just pass through
                new_annotations.append(ann1)
                new_points.append(points[i])

        points = np.array(new_points)
        kd_tree = cKDTree(points)
        remaining_annotations = new_annotations

    return remaining_annotations

def flatten_segmentation(segmentation):
    if isinstance(segmentation, list):
        if all(isinstance(item, list) for item in segmentation):
            return [coord for sublist in segmentation for coord in sublist]
        elif all(isinstance(item, (int, float)) for item in segmentation):
            return segmentation
    return None

def update_json_file(annotations, file_path, image_path):
    # We will no longer force a single category_id since annotations can have multiple categories
    with Image.open(image_path) as img:
        width, height = img.size
        file_name = os.path.basename(image_path)

    # Collect category IDs from annotations
    category_ids = sorted(set(ann['category_id'] for ann in annotations))
    # For simplicity, we do not have category names here, you might want to add them 
    # if you have them available. For now, just name them generically.
    categories = [{"id": cid, "name": f"category_{cid}", "supercategory": "none"} for cid in category_ids]

    coco_data = {
        "images": [{"id": 1, "width": width, "height": height, "file_name": file_name}],
        "annotations": [],
        "categories": categories
    }

    for idx, ann in enumerate(annotations, start=1):
        standardized_segmentation = flatten_segmentation(ann['segmentation'])
        if standardized_segmentation is not None:
            coco_ann = {
                "id": idx,
                "image_id": 1,
                "category_id": ann['category_id'],  # Use the annotation's own category_id
                "segmentation": [standardized_segmentation],
                "area": ann['area'],
                "bbox": ann['bbox'],
                "iscrowd": 0
            }
            coco_data['annotations'].append(coco_ann)
        else:
            print(f"Error in segmentation format for annotation {idx}, unable to process.")

    with open(file_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

def check_specific_annotations(annotations, id1, id2):
    # Retrieve the annotations by IDs
    ann1 = next((ann for ann in annotations if ann['annotation_id'] == id1), None)
    ann2 = next((ann for ann in annotations if ann['annotation_id'] == id2), None)

    if ann1 is None or ann2 is None:
        print(f"Annotation not found for given IDs: {id1}, {id2}")
        return
    
    # Convert points to polygons
    poly1 = create_polygon_from_points(ann1['polygon_points'])
    poly2 = create_polygon_from_points(ann2['polygon_points'])
    
    # Perform compatibility check
    is_compatible = check_compatibility_single(ann1, ann2)
    
    # Print details of the compatibility checks
    print(f"Checking compatibility between Annotation {id1} and Annotation {id2}:")
    print(f"Compatible: {is_compatible}")

    # Use pre-calculated principal axes lengths to compute the diagonal ratio
    major_axis_length_ann1 = ann1['principal_axes_lengths']['major_axis_length']
    minor_axis_length_ann1 = ann1['principal_axes_lengths']['minor_axis_length']
    diagonal_ratio_ann1 = minor_axis_length_ann1 / major_axis_length_ann1 if major_axis_length_ann1 > 0 else 0

    major_axis_length_ann2 = ann2['principal_axes_lengths']['major_axis_length']
    minor_axis_length_ann2 = ann2['principal_axes_lengths']['minor_axis_length']
    diagonal_ratio_ann2 = minor_axis_length_ann2 / major_axis_length_ann2 if major_axis_length_ann2 > 0 else 0

    # Check if diagonal ratio for either polygon exceeds 0.8
    ignore_orientation = diagonal_ratio_ann1 > 0.33 or diagonal_ratio_ann2 > 0.33    

    # Additional details
    intersection_area = calculate_intersection_area(poly1, poly2)
    overlap_length = calculate_line_overlap_length(poly1, poly2)
    dist = euclidean_distance(ann1['centroid'], ann2['centroid'])
    direct_diff, reverse_diff = angle_difference(ann1['orientation'], ann2['orientation'])
    orientation_diff = min(direct_diff, reverse_diff)
    convex_hull_ratio = convex_hull_area(np.array(ann1['polygon_points']), np.array(ann2['polygon_points'])) / combine_areas(ann1['area'], ann2['area'])
    dist_Threshold = ann1['principal_axes_lengths']['major_axis_length'] + ann2['principal_axes_lengths']['major_axis_length'] - overlap_length

    print(f"Intersection Area: {intersection_area}")
    print(f"Overlap Length: {overlap_length}")
    print('----------')
    print(f"Distance Thresholds: {dist_Threshold}")
    print(f"Distance Between Centroids: {dist}")
    print(f"Should be greater than Threshold")
    print('----------')
    print(f'Ignore Orientation:{ignore_orientation}')
    print(f'Orientation Threshold: 5 degrees')
    print(f"Orientation Difference: {orientation_diff}")
    print(f"Direct Difference: {direct_diff}")
    print(f"Reverse Difference: {reverse_diff}")
    print(f"Should be less than Threshold")
    print('----------')
    print(f"Convex Hull Threshold: 1.3")
    print(f"Convex Hull Ratio: {convex_hull_ratio}")
    print(f"Should be less than Threshold")

    # Optional: Visualize the annotations for visual inspection
    visualize_annotations(ann1, ann2)

In [ ]:
# Adjustment Functions
def add_annotations_to_coco_json(file1_path, file2_path, output_path):
    # Load the first file
    with open(file1_path, 'r') as f:
        data1 = json.load(f)

    # If the second file doesn't exist, just save the first file's data to output
    if not os.path.exists(file2_path):
        with open(output_path, 'w') as f:
            json.dump(data1, f, indent=4)
        return data1

    # Load the second file
    with open(file2_path, 'r') as f:
        data2 = json.load(f)

    # Ensure that both files have 'annotations', 'images', and 'categories' keys
    data1.setdefault('annotations', [])
    data1.setdefault('images', [])
    data1.setdefault('categories', [])
    data2.setdefault('annotations', [])
    data2.setdefault('images', [])
    data2.setdefault('categories', [])

    # Build dictionaries to map category names to IDs for data2
    data2_categories_by_name = {cat['name']: cat['id'] for cat in data2['categories']}
    # If no categories exist in data2, start fresh
    max_category_id = max([cat['id'] for cat in data2['categories']], default=0)

    # Re-map category_ids in data1 to match data2
    category_id_map = {}
    for cat in data1['categories']:
        cat_name = cat['name']
        if cat_name in data2_categories_by_name:
            # Category exists in data2
            category_id_map[cat['id']] = data2_categories_by_name[cat_name]
        else:
            # Need to add this category to data2
            max_category_id += 1
            new_cat_id = max_category_id
            data2['categories'].append({
                "id": new_cat_id,
                "name": cat_name,
                "supercategory": cat.get("supercategory", "none")
            })
            data2_categories_by_name[cat_name] = new_cat_id
            category_id_map[cat['id']] = new_cat_id

    # Now re-map the category_id of each annotation in data1 using category_id_map
    for annotation in data1['annotations']:
        old_cat_id = annotation['category_id']
        if old_cat_id in category_id_map:
            annotation['category_id'] = category_id_map[old_cat_id]
        else:
            # If not found, default to a known category or raise an error
            # Here we raise an error because every category should be mapped
            raise ValueError(f"No category mapping found for category_id {old_cat_id}")

    # Start IDs from the maximum found in data2 + 1 for annotations and images
    max_annotation_id = max([ann['id'] for ann in data2['annotations']], default=0) + 1
    max_image_id = max([img['id'] for img in data2['images']], default=0) + 1

    # Update image IDs in data1
    image_id_map = {}
    for image in data1['images']:
        original_id = image['id']
        new_id = max_image_id
        image_id_map[original_id] = new_id
        image['id'] = new_id
        max_image_id += 1

    # Update annotation IDs and their image references in data1
    for annotation in data1['annotations']:
        original_id = annotation['id']
        new_id = max_annotation_id
        annotation['id'] = new_id
        # Update image_id to the mapped ID
        if annotation['image_id'] in image_id_map:
            annotation['image_id'] = image_id_map[annotation['image_id']]
        else:
            raise ValueError(f"No image mapping found for image_id {annotation['image_id']}")

        max_annotation_id += 1

    # Append data1's annotations and images to data2
    data2['annotations'].extend(data1['annotations'])
    data2['images'].extend(data1['images'])
    # Categories are already merged and updated in data2

    # Save the updated data
    with open(output_path, 'w') as f:
        json.dump(data2, f, indent=4)

    return data2

def adjust_coco_annotations(coco_json_path, top_padding, bottom_padding, left_padding, right_padding, output_json_path):
    with open(coco_json_path) as file:
        coco_data = json.load(file)

    # Adjust image dimensions
    for image in coco_data['images']:
        image['height'] -= (top_padding + bottom_padding)
        image['width'] -= (left_padding + right_padding)

    # Adjust annotation coordinates
    for annotation in coco_data['annotations']:
        bbox = annotation['bbox']
        bbox[0] -= left_padding
        bbox[1] -= top_padding
        annotation['bbox'] = bbox

        if 'segmentation' in annotation:
            new_segmentation = []
            for segment in annotation['segmentation']:
                if isinstance(segment, list):
                    new_polygon = []
                    # Iterate in pairs
                    points = iter(segment)
                    for x, y in zip(points, points):
                        new_x = x - left_padding
                        new_y = y - top_padding
                        new_polygon.extend([new_x, new_y])
                    new_segmentation.append(new_polygon)
            annotation['segmentation'] = new_segmentation

    with open(output_json_path, 'w') as file:
        json.dump(coco_data, file, indent=4)


In [ ]:
# Statistics Functions
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

def load_coco_annotations(annotation_path):
    coco = COCO(annotation_path)
    return coco

def calc_bounds(mean, std_dev, upper_multiplier, lower_multiplier):
    return mean + upper_multiplier * std_dev, mean - lower_multiplier * std_dev

def filter_coordinates(x, y, z, rounds, background_value=0):
    filtered_x, filtered_y, filtered_z = cp.array(x), cp.array(y), cp.array(z)
    for round_details in rounds:
        mask = filtered_z > background_value
        filtered_x, filtered_y, filtered_z = filtered_x[mask], filtered_y[mask], filtered_z[mask]

        mean_z, std_dev_z = cp.mean(filtered_z), cp.std(filtered_z)
        upper_z, lower_z = calc_bounds(mean_z, std_dev_z, *round_details['z'])
        z_mask = (filtered_z <= upper_z) & (filtered_z >= lower_z)

        mean_x, std_dev_x = cp.mean(filtered_x[z_mask]), cp.std(filtered_x[z_mask])
        upper_x, lower_x = calc_bounds(mean_x, std_dev_x, *round_details['x'])
        x_mask = (filtered_x[z_mask] <= upper_x) & (filtered_x[z_mask] >= lower_x)

        mean_y, std_dev_y = cp.mean(filtered_y[z_mask][x_mask]), cp.std(filtered_y[z_mask][x_mask])
        upper_y, lower_y = calc_bounds(mean_y, std_dev_y, *round_details['y'])
        y_mask = (filtered_y[z_mask][x_mask] <= upper_y) & (filtered_y[z_mask][x_mask] >= lower_y)

        filtered_x, filtered_y, filtered_z = filtered_x[z_mask][x_mask][y_mask], filtered_y[z_mask][x_mask][y_mask], filtered_z[z_mask][x_mask][y_mask]

    return filtered_x, filtered_y, filtered_z

def trim_noise_based_on_coordinate(coordinate, z_values, upper_std_dev, lower_std_dev, background_value=0):
    unique_coords = cp.unique(coordinate)
    filtered_indices = []
    for unique_val in unique_coords:
        local_mask = coordinate == unique_val
        local_z_values = z_values[local_mask]
        if local_z_values.size == 0:
            continue

        local_mean, local_std = cp.mean(local_z_values), cp.std(local_z_values)
        upper, lower = calc_bounds(local_mean, local_std, upper_std_dev, lower_std_dev)
        
        valid_indices = cp.where((local_z_values <= upper) & (local_z_values >= max(lower, background_value)))[0]
        
        if valid_indices.size > 0:
            filtered_indices.append(valid_indices)
    
    if not filtered_indices:
        return cp.array([])
    
    return cp.concatenate(filtered_indices)

def filter_gaussian_test(x, y, z, final_adjust=0.66, background_value=0):
    rounds = [
        {'z': (3, 1), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.25, 1), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.5, 1), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.75, 1), 'x': (2, 2), 'y': (2, 2)},
        {'z': (4, 1), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.85, 0.96), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.7, 0.92), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.55, 0.88), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.4, 0.84), 'x': (2, 2), 'y': (2, 2)},
        {'z': (3.25, 0.8), 'x': (1.5, 1.5), 'y': (1.5, 1.5)},
    ]
    
    x_filtered, y_filtered, z_filtered = filter_coordinates(x, y, z, rounds, background_value)

    filtered_indices_x = trim_noise_based_on_coordinate(x_filtered, z_filtered, final_adjust, 5 * final_adjust)
    filtered_indices_y = trim_noise_based_on_coordinate(y_filtered, z_filtered, final_adjust, 5 * final_adjust)
    
    if filtered_indices_x.size > 0 and filtered_indices_y.size > 0:
        intersection_indices = cp.intersect1d(filtered_indices_x, filtered_indices_y)
    else:
        intersection_indices = cp.array([])

    max_z = cp.max(z_filtered[intersection_indices]) if intersection_indices.size > 0 else None
    return max_z

def extract_surface_data(coco, image_path, annotation_id):
    annotation = coco.loadAnns(annotation_id)[0]
    image = Image.open(image_path)
    
    mask = Image.new('L', (image.width, image.height), 0)
    if 'segmentation' in annotation and annotation['segmentation']:
        for segment in annotation['segmentation']:
            ImageDraw.Draw(mask).polygon(segment, outline=1, fill=1)
    mask = cp.array(mask)

    region = cp.where(mask == 1)
    image_np = cp.array(image)
    pixel_values = image_np[region[0], region[1]] if image_np.ndim == 2 else image_np[region[0], region[1], 0]
    
    return region[1], region[0], pixel_values

def calculate_centroid_seg(segmentation):
    x = cp.array(segmentation[0::2])
    y = cp.array(segmentation[1::2])
    return float(cp.mean(x)), float(cp.mean(y))

def calculate_principal_diagonal_from_segmentation(segmentation):
    x, y = cp.array(segmentation[0::2]), cp.array(segmentation[1::2])
    coordinates = cp.vstack((x, y)).T
    pca = PCA(n_components=1)
    # PCA in sklearn needs CPU arrays
    pca.fit(coordinates.get()) 
    projections = cp.dot(coordinates, cp.array(pca.components_[0]))
    return float(cp.max(projections) - cp.min(projections))

def create_nan_masked_image(coco, image_path):
    image = Image.open(image_path)
    image_np = cp.array(image, dtype=cp.float32)
    overlap_mask = cp.zeros((image_np.shape[0], image_np.shape[1]), dtype=cp.uint8)
    
    for annotation in coco.anns.values():
        mask = Image.new('L', (image.width, image.height), 0)
        if 'segmentation' in annotation and annotation['segmentation']:
            for segment in annotation['segmentation']:
                ImageDraw.Draw(mask).polygon(segment, outline=1, fill=1)
        overlap_mask += cp.array(mask)

    image_nan_masked = cp.where(overlap_mask > 1, cp.nan, image_np)
    return image_nan_masked

def process_annotation(annotation, coco, image_path, image_nan_masked):
    annotation_id = annotation['id']
    category_id = annotation.get('category_id', 1)  # Make sure category_id is recorded

    x, y, z = extract_surface_data(coco, image_path, annotation_id)

    y_np = y.get() if isinstance(y, cp.ndarray) else y
    x_np = x.get() if isinstance(x, cp.ndarray) else x

    z_nan_masked = cp.array([image_nan_masked[y_np[i], x_np[i]] for i in range(len(x_np))])

    segmentation = annotation['segmentation'][0]
    centroid_x, centroid_y = calculate_centroid_seg(segmentation)
    principal_diagonal = calculate_principal_diagonal_from_segmentation(segmentation)

    fitted_max_value = filter_gaussian_test(x, y, z_nan_masked, final_adjust=0.66, background_value=0)
    fitted_max_value_nm = fitted_max_value * conversion_factor if fitted_max_value is not None else None

    return {
        'Annotation ID': annotation_id,
        'Category ID': category_id,
        'Centroid X': centroid_x,
        'Centroid Y': centroid_y,
        'Principal Diagonal': principal_diagonal,
        'Fitted Max Value': fitted_max_value,
        'Fitted Max Value (nm)': fitted_max_value_nm
    }


In [ ]:
#Visualization Functions
def create_black_duplicate(image_path):
    with Image.open(image_path) as img:
        img_array = np.array(img)
        shape = img_array.shape
        black_array = np.zeros(shape, dtype=img_array.dtype)
        black_image = Image.fromarray(black_array)
        base, extension = os.path.splitext(image_path)
        black_image_path = f"{base}_black{extension}"
        black_image.save(black_image_path)
        return black_image_path

def get_color_from_colormap(value, min_value, max_value):
    if isinstance(min_value, cp.ndarray):
        min_value = min_value.get()
    if isinstance(max_value, cp.ndarray):
        max_value = max_value.get()
    if isinstance(value, cp.ndarray):
        value = value.get()

    norm = mcolors.Normalize(vmin=min_value, vmax=max_value)
    cmap = cm.rainbow
    rgba_color = cmap(norm(value))
    color = tuple(int(255 * x) for x in rgba_color[:3])
    return color

def visualize_nodes_on_image_with_colormap(image_path, df_clean, output_path, black=False):
    if black:
        image_path = create_black_duplicate(image_path)

    image = Image.open(image_path).convert("RGBA")
    draw = ImageDraw.Draw(image)

    min_value = df_clean['Fitted Max Value'].min()
    max_value = df_clean['Fitted Max Value'].max()

    norm_values = (df_clean['Fitted Max Value'] - min_value) / (max_value - min_value)

    # Define shapes for categories
    category_shapes = {
        1: 'ellipse',
        2: 'rectangle'
        # Add more mappings as needed, e.g., 3: 'triangle'
    }

    for idx, row in df_clean.iterrows():
        x, y = row['Centroid X'], row['Centroid Y']
        value = row['Fitted Max Value']
        category_id = row.get('Category ID', 1)  # Default to 1 if not present
        color = get_color_from_colormap(value, min_value, max_value)

        norm_value = norm_values.iloc[idx]
        size = 10 + 20 * norm_value
        bbox = [x - size, y - size, x + size, y + size]

        shape = category_shapes.get(category_id, 'ellipse')  # Default to ellipse
        if shape == 'ellipse':
            draw.ellipse(bbox, fill=color)
        elif shape == 'rectangle':
            draw.rectangle(bbox, fill=color)
        else:
            # If you add a 'triangle' shape:
            # Triangles need three points, for example:
            # triangle_points = [(x, y - size), (x - size, y + size), (x + size, y + size)]
            # draw.polygon(triangle_points, fill=color)
            draw.ellipse(bbox, fill=color)  # fallback to ellipse

    image = image.convert("RGB")
    image.save(output_path)

    plt.figure(figsize=(30, 30))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def visualize_and_connect_nodes(image_path, df_clean, output_path, black=False):
    if black:
        image_path = create_black_duplicate(image_path)

    image = Image.open(image_path).convert("RGBA")
    draw = ImageDraw.Draw(image)

    min_value = df_clean['Fitted Max Value'].min()
    max_value = df_clean['Fitted Max Value'].max()

    coords = df_clean[['Centroid X', 'Centroid Y']].values
    nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(coords)
    distances, indices = nbrs.kneighbors(coords)

    df_clean['Connections'] = [list(ind[1:]) for ind in indices]

    category_shapes = {
        1: 'ellipse',
        2: 'rectangle'
        # Add more shapes if needed
    }

    for idx, row in df_clean.iterrows():
        x, y = row['Centroid X'], row['Centroid Y']
        value = row['Fitted Max Value']
        category_id = row.get('Category ID', 1)
        color = get_color_from_colormap(value, min_value, max_value)

        # Draw connections first
        for neighbor_idx in row['Connections']:
            neighbor_x = df_clean.loc[neighbor_idx, 'Centroid X']
            neighbor_y = df_clean.loc[neighbor_idx, 'Centroid Y']
            draw.line((x, y, neighbor_x, neighbor_y), fill=color, width=2)

        size = 10 + 20 * ((value - min_value) / (max_value - min_value))
        bbox = [x - size, y - size, x + size, y + size]

        shape = category_shapes.get(category_id, 'ellipse')
        if shape == 'ellipse':
            draw.ellipse(bbox, fill=color)
        elif shape == 'rectangle':
            draw.rectangle(bbox, fill=color)
        else:
            # fallback if needed
            draw.ellipse(bbox, fill=color)

    image = image.convert("RGB")
    image.save(output_path)

    plt.figure(figsize=(30, 30))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def create_colorbar(min_value, max_value, colormap):
    if isinstance(min_value, cp.ndarray):
        min_value = min_value.get()
    if isinstance(max_value, cp.ndarray):
        max_value = max_value.get()

    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.5)

    norm = mcolors.Normalize(vmin=min_value, vmax=max_value)
    cb = cm.ScalarMappable(norm=norm, cmap=colormap)
    cb.set_array([])
    cbar = plt.colorbar(cb, orientation='horizontal', cax=ax)
    cbar.set_label('Slip Intensity Values (nm)')
    plt.show()

def visualize_coco_masks_bbox_black(image_path, coco_json_path, output_path, figure_size=(50, 50), font_size=20, black=False, bbox=False):
    if black:
        image_path = create_black_duplicate(image_path)
    
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    image = Image.open(image_path).convert('RGB')
    draw = ImageDraw.Draw(image)

    try:
        font = ImageFont.truetype("arial.ttf", size=font_size)
    except IOError:
        font = ImageFont.load_default()

    # Define a category color mapping (example: 2 classes)
    category_colors = {
        1: (255, 0, 0),   # Red for Category 1
        2: (0, 255, 0)    # Green for Category 2
    }

    for annotation in coco_data["annotations"]:
        segmentation = annotation["segmentation"][0]
        category_id = annotation.get("category_id", 1)
        
        # Use category_colors if category_id exists, else default
        color = category_colors.get(category_id, (255, 255, 255))  # Default to white if not found

        draw.polygon(segmentation, outline=color, fill=None)
        centroid_x = sum(segmentation[::2]) / (len(segmentation) / 2)
        centroid_y = sum(segmentation[1::2]) / (len(segmentation) / 2)
        draw.text((centroid_x + 5, centroid_y), str(annotation['id']), fill=color, font=font)

        if bbox:
            bbox_coords = [
                annotation["bbox"][0], annotation["bbox"][1],
                annotation["bbox"][0] + annotation["bbox"][2],
                annotation["bbox"][1] + annotation["bbox"][3]
            ]
            draw.rectangle(bbox_coords, outline=color, width=2)

    plt.figure(figsize=figure_size)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    image.save(output_path)
    

# Code Execution

In [ ]:
# Increase the maximum number of pixels that can be loaded.
Image.MAX_IMAGE_PIXELS = None  # This disables the decompression bomb check entirely.
# Alternatively, set a new limit that's specific to your image size, if known and reasonable

start_time = time.perf_counter()

# Specify which categories to merge
# For example, merge only category_id=1:
categories_to_merge = [1]  
# If you have multiple classes and you want to merge only a subset, list them here.

for image_path in image_paths:
    output_folder = Path(base_output_folder) / Path(image_path).stem
    os.makedirs(output_folder, exist_ok=True)

    #--------------------------- Initial Predictions -------------------------------------------------
    raw_image_path = image_path

    #Find previous Annotations from prior Steps
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    previous_json_path = os.path.join(os.path.dirname(image_path), f"{base_name}_predictions.json")
    print(previous_json_path)

    #Preprocess the Image
    raw_image_path = apply_previous_predictions(image_path, previous_json_path, output_folder)
    preprocessed_image_path = preprocess(raw_image_path, output_folder)
    preprocessed_original_image_path = preprocess(image_path, output_folder)

    #Pad the Image
    padded_image_path, top_padding, bottom_padding, left_padding, right_padding = pad_image(preprocessed_image_path, output_folder, min_padding=256)
    final_image_path = padded_image_path
    image_name = os.path.basename(final_image_path)

    # Loop through the steps for counters 0 to 64 (currently set to 10 for demonstration)
    for counter in range(10):
        # Slice the Image
        shift_distance = 128
        slice_folder_path = slice_image_to_1024(padded_image_path, output_folder, counter, shift_distance)

        # Predict on Image
        model = YOLO(model_path)
        model.to('cuda')
        output_json_path = process_images(slice_folder_path, output_folder, model, counter, min_size=200, area_threshold=30)

        # Convert to large image
        output_file_path = convert_coco_to_large_image(output_json_path, output_folder, padded_image_path, counter)

        # Mask the Image
        masked_image_path = mask_image_with_coco_masks(padded_image_path, output_file_path, output_folder, counter, dilation_distance=5)

        # Update the padded_image_path for the next iteration
        padded_image_path = masked_image_path

    output_json = 'Final_Compiled_Predictions.json'
    final_compiled_predictions_path = compile_coco_json(output_folder, output_json, image_name)

    #--------------------------- Visualize Initial Predictions -------------------------------------------------
    initial_masked_image_path = os.path.join(output_folder, "Initial_Masked_Image.jpg")
    visualize_coco_masks(final_image_path, final_compiled_predictions_path, initial_masked_image_path, figure_size=(100, 100), font_size=20)

    #-------------------------- Remove Padding from the Images --------------------------------------------------
    coco_json_path = final_compiled_predictions_path
    file_name = 'Final_Compiled_Predictions_Adjusted.json'
    adjusted_predictions_path = os.path.join(output_folder, file_name)
    adjust_coco_annotations(coco_json_path, top_padding, bottom_padding, left_padding, right_padding, adjusted_predictions_path)

    #-------------------------- Add in Previous Predictions -------------------------------------------------
    full_adjusted_predictions_path = os.path.join(output_folder, 'Final_Compiled_Predictions_Cleaned.json')
    add_annotations_to_coco_json(adjusted_predictions_path, previous_json_path, full_adjusted_predictions_path)

    masked_image_path = os.path.join(output_folder, "Masked_Image.jpg")
    visualize_coco_masks(preprocessed_original_image_path, full_adjusted_predictions_path, masked_image_path, figure_size=(100, 100), font_size=20)
    
    #--------------------------- Clean Initial Predictions -------------------------------------------------
    clean_annotations(preprocessed_original_image_path, full_adjusted_predictions_path, full_adjusted_predictions_path, intensity_threshold=1)

    clean_masked_image_path = os.path.join(output_folder, "Clean_Masked_Image.jpg")
    visualize_coco_masks(preprocessed_original_image_path, full_adjusted_predictions_path, clean_masked_image_path, figure_size=(100, 100), font_size=20)

    #--------------------------- Merge Predictions -------------------------------------------------
    full_adjusted_cleaned_predictions_path = os.path.join(output_folder, 'Final_Compiled_Predictions_Cleaned_Adjusted.json')

    # Load initial annotations
    start_time_merging = time.perf_counter()

    annotations = process_coco_json(full_adjusted_predictions_path)
    print('Intial Number of Features:', len(annotations))

    # Pass categories_to_merge to only merge specified categories
    final_annotations = progressive_merge_annotations(annotations, categories_to_merge=categories_to_merge)
    print('Final Number of Features:', len(final_annotations))

    # Because update_json_file no longer takes a single category_id, we no longer pass category_id directly here.
    # The final_annotations now contain multiple categories. They will be preserved as is.
    update_json_file(final_annotations, full_adjusted_cleaned_predictions_path, preprocessed_original_image_path)

    print("Combination complete. Updated annotations saved.")

    end_time_merging = time.perf_counter()
    elapsed_seconds = end_time_merging - start_time_merging
    hours, minutes, seconds = convert_seconds_to_hms(elapsed_seconds)
    print(f"Execution time for Merging: {hours} hour(s), {minutes} minute(s), {seconds:.2f} second(s)")

    #-------------------------- Visualize the Merged Predictions -------------------------------------------------
    final_masked_image_cleaned_path = os.path.join(output_folder, "Initial_Masked_Image_Cleaned.jpg")
    visualize_coco_masks(preprocessed_original_image_path, full_adjusted_cleaned_predictions_path, final_masked_image_cleaned_path ,figure_size=(100, 100), font_size=20)

    final_masked_image_cleaned_bbox_path = os.path.join(output_folder, "Initial_Masked_Image_Cleaned_bbox.jpg")
    visualize_coco_masks_bbox(preprocessed_original_image_path, full_adjusted_cleaned_predictions_path, final_masked_image_cleaned_bbox_path ,figure_size=(100, 100), font_size=20)

    #-------------------------- Statistics Postprocessing --------------------------------------------------
    start_time_stats = time.perf_counter()
    data = load_json(full_adjusted_cleaned_predictions_path)
    coco = load_coco_annotations(full_adjusted_cleaned_predictions_path)
    data_list = []

    NaN_Image_name = 'nan_masked_image.tif'
    mask_save_path = os.path.join(output_folder, NaN_Image_name)

    # Create or load the NaN-masked image
    image_nan_masked = tiff.imread(mask_save_path) if os.path.exists(mask_save_path) else create_nan_masked_image(coco, image_path)
    if not os.path.exists(mask_save_path):
        tiff.imwrite(mask_save_path, image_nan_masked.get(), dtype=np.float32)

    # Collect Annotations 
    annotation_ids_to_analyze = []
    annotations = data['annotations'] if not annotation_ids_to_analyze else [
        annotation for annotation in data['annotations'] if annotation['id'] in annotation_ids_to_analyze
    ]

    # Parallel processing of annotations with concurrent.futures
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_annotation, annotation, coco, image_path, image_nan_masked) for annotation in annotations]
        data_list = [future.result() for future in concurrent.futures.as_completed(futures)]

    # Convert CuPy arrays in data_list to NumPy if necessary
    data_list = [
        {k: (v.get() if isinstance(v, cp.ndarray) else v) for k, v in item.items()}
        for item in data_list
    ]

    # Save the DataFrame to a CSV file
    filename = 'BlN_Stats.csv'
    BLN_Stats_Path = os.path.join(output_folder, filename)
    df = pd.DataFrame(data_list)
    df = df.sort_values(by='Annotation ID')
    df.to_csv(BLN_Stats_Path, index=False)

    end_time_stats = time.perf_counter()
    elapsed_seconds = end_time_stats - start_time_stats
    hours, minutes, seconds = convert_seconds_to_hms(elapsed_seconds)
    print(f"Execution time for Statistics: {hours} hour(s), {minutes} minute(s), {seconds:.2f} second(s)")

    #------------------------- Node Visualization ---------------------------------------------------
    df_clean = df.dropna(subset=['Centroid X', 'Centroid Y', 'Fitted Max Value'])
    df_clean = df_clean.reset_index(drop=True)

    # Visualize nodes (with shapes based on category)
    filename = 'Node_Overlay.jpg'
    output_path = f'{output_folder}/{filename}'
    visualize_nodes_on_image_with_colormap(preprocessed_original_image_path, df_clean, output_path, black=True)

    #------------------------- Node Visualization with Connections --------------------------------------------------
    df_clean = df.dropna(subset=['Centroid X', 'Centroid Y', 'Fitted Max Value'])
    df_clean = df_clean.reset_index(drop=True)
    
    filename = 'Node_Overlay_with_Connections.jpg'
    output_path = f'{output_folder}/{filename}'
    visualize_and_connect_nodes(preprocessed_original_image_path, df_clean, output_path, black=False)
    filename = 'Node_Overlay_with_Connections_Black.jpg'
    output_path = f'{output_folder}/{filename}'
    visualize_and_connect_nodes(preprocessed_original_image_path, df_clean, output_path, black=True)

    #------------------------- Display the colorbar ---------------------------------------------------
    min_value = df_clean['Fitted Max Value'].min() * 22.46
    max_value = df_clean['Fitted Max Value'].max() * 22.46
    create_colorbar(min_value, max_value, cm.rainbow)

    #------------------------- Final Visualizations --------------------------------------------------
    final_masked_image_cleaned_adjusted_path = os.path.join(output_folder, "Final_Masked_Image_Cleaned_Adjusted.jpg")
    visualize_coco_masks_bbox_black(preprocessed_original_image_path, full_adjusted_cleaned_predictions_path, final_masked_image_cleaned_adjusted_path ,figure_size=(100, 100), font_size=20, black=False, bbox=False)

    final_masked_image_cleaned_adjusted_bbox_path = os.path.join(output_folder, "Final_Masked_Image_Cleaned_Adjusted_bbox.jpg")
    visualize_coco_masks_bbox_black(preprocessed_original_image_path, full_adjusted_cleaned_predictions_path, final_masked_image_cleaned_adjusted_bbox_path ,figure_size=(100, 100), font_size=20, black=False, bbox=True)

    final_masked_image_cleaned_adjusted_black_path = os.path.join(output_folder, "Final_Masked_Image_Cleaned_Adjusted_black.jpg")
    visualize_coco_masks_bbox_black(preprocessed_original_image_path, full_adjusted_cleaned_predictions_path, final_masked_image_cleaned_adjusted_black_path ,figure_size=(100, 100), font_size=20, black=True, bbox=False)

    final_masked_image_cleaned_adjusted_bbox_black_path = os.path.join(output_folder, "Final_Masked_Image_Cleaned_Adjusted_bbox_black.jpg")
    visualize_coco_masks_bbox_black(preprocessed_original_image_path, full_adjusted_cleaned_predictions_path, final_masked_image_cleaned_adjusted_bbox_black_path ,figure_size=(100, 100), font_size=20, black=True, bbox=True)

    #------------------------- Create Histogram --------------------------------------------------
    df_clean['Converted Max Value'] = df_clean['Fitted Max Value'] * conversion_factor

    plt.figure(figsize=(8, 6))
    sns.violinplot(y='Converted Max Value', data=df_clean, inner=None, color='red')
    sns.stripplot(y='Converted Max Value', data=df_clean, size=4, jitter=True, color='k', edgecolor='k', alpha=0.7)

    ax = plt.gca()
    violins = [child for child in ax.get_children() if isinstance(child, PolyCollection)]
    for violin in violins:
        paths = violin.get_paths()
        for path in paths:
            vertices = path.vertices
            median = np.median(vertices[:, 0])
            vertices[vertices[:, 0] > median, 0] = median

    plt.title('Distribution of Maximum Slip Intensity Across the Dataset')
    plt.ylabel('Max Slip Intensity (nm)')
    plt.tight_layout()
    plt.show()

    #------------------------- Organize Files ---------------------------------------------------
    base_dir = output_folder

    clean_results_files = [
        "Final_Compiled_Predictions_Cleaned_Adjusted.json",
        "Final_Masked_Image_Cleaned_Adjusted.jpg",
        "Final_Masked_Image_Cleaned_Adjusted_bbox.jpg",
        "Final_Masked_Image_Cleaned_Adjusted_black.jpg",
        "Final_Masked_Image_Cleaned_Adjusted_bbox_black.jpg",
        "BlN_Stats.csv",
        "Node_Overlay_with_Connections.jpg",
        "Node_Overlay_with_Connections_Black.jpg"
    ]

    clean_results_dir = os.path.join(base_dir, 'Clean_Results')
    os.makedirs(clean_results_dir, exist_ok=True)

    processing_files_dir = os.path.join(base_dir, 'Processing_Files')
    os.makedirs(processing_files_dir, exist_ok=True)

    for file_name in clean_results_files:
        source_path = os.path.join(base_dir, file_name)
        if os.path.exists(source_path):
            shutil.move(source_path, os.path.join(clean_results_dir, file_name))

    for item in os.listdir(base_dir):
        item_path = os.path.join(base_dir, item)
        if item_path not in [clean_results_dir, processing_files_dir]:
            shutil.move(item_path, os.path.join(processing_files_dir, item))

    json_file_path = os.path.join(clean_results_dir, "Final_Compiled_Predictions_Cleaned_Adjusted.json")
    if os.path.exists(json_file_path):
        base_name = Path(image_path).stem
        match = re.search(r'(\d+)(?!.*\d)', base_name)
        if match:
            number = int(match.group(0)) + 1
            new_base_name = re.sub(r'(\d+)(?!.*\d)', str(number), base_name)
            new_file_name = f"{new_base_name}_predictions.json"
            
            destination_path = os.path.join(base_output_folder, new_file_name)
            shutil.copy(json_file_path, destination_path)

    print("Files have been organized.")

    end_time = time.perf_counter()
    elapsed_seconds = end_time - start_time
    hours, minutes, seconds = convert_seconds_to_hms(elapsed_seconds)
    print(f"Execution time: {hours} hour(s), {minutes} minute(s), {seconds:.2f} second(s)")
